In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import load_model

In [2]:
ASJC_codes = pd.read_csv("ASJC_label_index_201012.csv")
ASJC_codes.head()

ASJC  index
0  1000.0      0
1  1100.0      1
2  1101.0      2
3  1102.0      3
4  1103.0      4

In [3]:
file_name_score = "scopus_data_model_201012_test_seed_123_min_20.csv"
df = pd.read_csv(file_name_score)
df.head()

FIRST_ASJC          EID                         DOI  ASJC  \
0      1000.0  85082380520  10.1038/s41598-020-62127-3  1000   
1      1000.0  85080101682  10.1038/s41598-020-59770-1  1000   
2      1000.0  85081197743  10.1038/s41598-020-60939-x  1000   
3      1000.0  85081745260  10.1038/s41598-020-61425-0  1000   
4      1000.0  85082931755     10.1126/science.aaz1183  1000   

                                               TandA  
0  DNA methylation ambiguity in the Fibrillin-1 (...  
1  Influences of pH on transport of arsenate (As5...  
2  Theileria parasites subvert E2F signaling to s...  
3  Heme b distributions through the Atlantic Ocea...  
4  A slip law for glaciers on deformable beds Sli...

In [4]:
df = pd.merge(left = df, right = ASJC_codes, how = "left",
              left_on = "FIRST_ASJC", right_on = "ASJC", )
df = df.rename(columns = {"ASJC_x": "ASJC"})
df.head()

FIRST_ASJC          EID                         DOI  ASJC  \
0      1000.0  85082380520  10.1038/s41598-020-62127-3  1000   
1      1000.0  85080101682  10.1038/s41598-020-59770-1  1000   
2      1000.0  85081197743  10.1038/s41598-020-60939-x  1000   
3      1000.0  85081745260  10.1038/s41598-020-61425-0  1000   
4      1000.0  85082931755     10.1126/science.aaz1183  1000   

                                               TandA  ASJC_y  index  
0  DNA methylation ambiguity in the Fibrillin-1 (...  1000.0      0  
1  Influences of pH on transport of arsenate (As5...  1000.0      0  
2  Theileria parasites subvert E2F signaling to s...  1000.0      0  
3  Heme b distributions through the Atlantic Ocea...  1000.0      0  
4  A slip law for glaciers on deformable beds Sli...  1000.0      0

In [5]:
token_text = df["TandA"]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(token_text)
sequences = tokenizer.texts_to_sequences(token_text)

In [6]:
max_len = 500
X_test  = pad_sequences(sequences, maxlen = max_len)
y_test  = to_categorical(df["index"].values)

In [7]:
file_name_model = "model_ASJC_1000_best_201012.h5"
loaded_model = load_model(file_name_model)
pred = loaded_model.predict(X_test, use_multiprocessing = True)

In [8]:
ASJC_top_n = 3 # 출력 ASJC코드 개수 지정용 변수
colname_ASJC_top = "ASJC_top" # ASJC코드가 확률이 높은순서대로 정렬되어 있음. 변경 X
colname_ASJC_pred_prob = "ASJC_pred_prob"# 정렬된 ASJC코드에 해당하는 확률값이 있음.변경 X

In [34]:
df_x_test = pd.DataFrame({"ASJC": ASJC_codes["ASJC"],
                          "prob": pred[0]})
df_x_test = df_x_test.sort_values("prob", ascending = False)
df_x_test["prob"] = df_x_test["prob"].astype("str").str.slice(start = 0, stop = 5)
df_x_test.head()

ASJC   prob
208  2736.0  0.650
322  3611.0  0.062
209  2737.0  0.044
217  2745.0  0.035
181  2708.0  0.033

In [35]:
def prop_top_n(x, top = ASJC_top_n):
    df_x = pd.DataFrame({"ASJC": ASJC_codes["ASJC"],
                         "prob": x})
    df_x = df_x.sort_values("prob", ascending = False)
    df_x = df_x.head(top).reset_index(drop = True)
    str_ASJC = df_x["ASJC"].astype("int").astype("str").str.cat(sep = "/")
    str_prob = df_x["prob"].astype("str").str.slice(start = 0, stop = 5).str.cat(sep = "/")
    return str_ASJC, str_prob

In [36]:
df[[colname_ASJC_top, colname_ASJC_pred_prob]] = pd.DataFrame(pred).apply(prop_top_n, axis = 1, result_type = "expand")

In [37]:
df.head(2)

FIRST_ASJC          EID                         DOI  ASJC  \
0      1000.0  85082380520  10.1038/s41598-020-62127-3  1000   
1      1000.0  85080101682  10.1038/s41598-020-59770-1  1000   

                                               TandA  ASJC_y  index  \
0  DNA methylation ambiguity in the Fibrillin-1 (...  1000.0      0   
1  Influences of pH on transport of arsenate (As5...  1000.0      0   

         ASJC_top     ASJC_pred_prob  
0  2736/3611/2737  0.650/0.062/0.044  
1  2701/3611/3004  0.788/0.052/0.035